# Using Optimization Techniques to Generate Winning FanDuel Lineups
___
Until I build one from scratch I'm going to be testing out `pydfs-lineup-optimizer` to generate optimal lineups with projections that I already have. More information about `pydfs-lineup-optimizer` can be found [here](https://github.com/DimaKudosh/pydfs-lineup-optimizer/tree/master/pydfs_lineup_optimizer/sites/fanduel).

**Here is one quick example of generating lineups based on average points per game.**

In [13]:
import pandas as pd
from pydfs_lineup_optimizer import get_optimizer, Site, Sport, CSVLineupExporter

In [14]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('player-lists/Week-11/FanDuel-NFL-2018-11-18-29988-players-list.csv')
for lineup in optimizer.optimize(n=5, max_exposure=0.3):
    print(lineup.players)
    print(lineup.fantasy_points_projection)
    print(lineup.salary_costs)

[Ryan Fitzpatrick QB (TB), Alvin Kamara RB (NO), T.J. Yeldon RB (JAC), John Brown WR (BAL), Emmanuel Sanders WR (DEN), JuJu Smith-Schuster WR (PIT), Eric Ebron TE (IND), Washington Redskins D (WAS), Melvin Gordon III RB (LAC)]
148.415
60000.0
[Matt Ryan QB (ATL), Alvin Kamara RB (NO), T.J. Yeldon RB (JAC), John Brown WR (BAL), Emmanuel Sanders WR (DEN), DeSean Jackson WR (TB), Eric Ebron TE (IND), Washington Redskins D (WAS), Melvin Gordon III RB (LAC)]
148.348
59900.0
[Ryan Fitzpatrick QB (TB), Saquon Barkley RB (NYG), Joe Mixon RB (CIN), Dede Westbrook WR (JAC), Michael Thomas WR (NO), DeSean Jackson WR (TB), Jared Cook TE (OAK), Denver Broncos D (DEN), Marlon Mack RB (IND)]
136.537
59900.0
[Ben Roethlisberger QB (PIT), Saquon Barkley RB (NYG), Nyheim Hines RB (IND), Dede Westbrook WR (JAC), Michael Thomas WR (NO), Calvin Ridley WR (ATL), Zach Ertz TE (PHI), Denver Broncos D (DEN), Marlon Mack RB (IND)]
134.902
60000.0
[Ben Roethlisberger QB (PIT), Nyheim Hines RB (IND), Joe Mixon RB

## Now instead of just using the average points per game, we'll bring in our other projections that *should* be more accurate.

___
#### Ballers first:

In [15]:
current_week = 11

qbs = pd.read_csv('projections/Week-{}/DFS - Fanduel QB.csv'.format(current_week))
rbs = pd.read_csv('projections/Week-{}/DFS - Fanduel RB.csv'.format(current_week))
wrs = pd.read_csv('projections/Week-{}/DFS - Fanduel WR.csv'.format(current_week))
tes = pd.read_csv('projections/Week-{}/DFS - Fanduel TE.csv'.format(current_week))
dst = pd.read_csv('projections/Week-{}/DFS - Fanduel DST.csv'.format(current_week))
full_team_names = list(dst.sort_values('Player')['Player'])

In [16]:
qbs.head(2)

,Player,Opp,Pts,Salary,Pts Per 1k
0,Jared Goff,vs. KC,27.3,"8,300",3.3
1,Drew Brees,vs. PHI,26.2,"8,500",3.1


In [17]:
footballers = pd.concat([qbs,rbs,wrs,tes,dst])
footballers.rename(columns={"Player": "Nickname"}, inplace=True)

___
#### Sportsline second:

In [18]:
sportsline = pd.read_csv('projections/Week-{}/projections.csv'.format(current_week))
sportsline.rename(columns={"PLAYER": "Nickname"}, inplace=True)
abbv_team_names = list(sportsline.loc[sportsline.POS == 'DEF', :].sort_values('Nickname')['Nickname'])
defenses_dict = dict(zip(abbv_team_names, full_team_names))
sportsline['Nickname'].replace(defenses_dict, inplace = True)
sportsline.head(2)

,Nickname,POS,TEAM,OPP,FP,fd exp,dk exp,PPR,CONS%,FREB%,PASSYD,RUSHYD,RECYD
0,Patrick Mahomes II,QB,KC,LAR,28.8,0.0,26.4,28.8,56%,33%,311.0,18.0,NaN
1,Cam Newton,QB,CAR,@DET,27.2,21.0,23.1,27.2,46%,53%,287.0,42.0,NaN


___
#### Averaging the projections:

In [19]:
total_projections = pd.merge(footballers, sportsline, on="Nickname")
total_projections['Salary'] = total_projections.Salary.apply(lambda x: int(x.replace(',', '')))
total_projections['Pts'] = (total_projections.Pts + total_projections.FP) / 2
total_projections['Pts Per 1k'] = total_projections['Pts'] / (total_projections.Salary / 1000)
total_projections.drop(labels=list(total_projections.columns[5:]), axis=1, inplace=True)
total_projections.head(3)

,Nickname,Opp,Pts,Salary,Pts Per 1k
0,Jared Goff,vs. KC,26.05,8300,3.138554
1,Drew Brees,vs. PHI,25.90,8500,3.047059
2,Carson Wentz,at NO,23.70,7700,3.077922


___
#### Bringing in the player list from FanDuel:

In [20]:
# Make sure you're selecting the correct file:
player_list=pd.read_csv('player-lists/Week-11/FanDuel-NFL-2018-11-18-29988-players-list.csv')
player_list.head(3)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
0,29988-42104,RB,Alvin,Alvin Kamara,Kamara,24.155555,9,8900,PHI@NO,NO,PHI,NaN,NaN,NaN,NaN,NaN
1,29988-64401,RB,Saquon,Saquon Barkley,Barkley,20.733334,9,8800,TB@NYG,NYG,TB,NaN,NaN,NaN,NaN,NaN
2,29988-28573,WR,Michael,Michael Thomas,Thomas,19.111111,9,8800,PHI@NO,NO,PHI,NaN,NaN,NaN,NaN,NaN


> Merging those 2 and making the swap:

In [21]:
master_projections = pd.merge(left=player_list, right=total_projections, how='inner', on='Nickname', suffixes=['_FD', '_tp'])
master_projections['FPPG'] = master_projections['Pts Per 1k'] # Experiment with Pts OR Pts Per 1k
master_projections.drop(['Opp', 'Pts', 'Salary_tp', 'Pts Per 1k'],axis=1, inplace=True)
master_projections.rename(columns={'Salary_FD':'Salary'}, inplace=True)
master_projections.to_csv('projections/Week-{}/master_projections.csv'.format(current_week))
master_projections.head(3)

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15
0,29988-42104,RB,Alvin,Alvin Kamara,Kamara,2.179775,9,8900,PHI@NO,NO,PHI,NaN,NaN,NaN,NaN,NaN
1,29988-64401,RB,Saquon,Saquon Barkley,Barkley,2.602273,9,8800,TB@NYG,NYG,TB,NaN,NaN,NaN,NaN,NaN
2,29988-28573,WR,Michael,Michael Thomas,Thomas,2.494318,9,8800,PHI@NO,NO,PHI,NaN,NaN,NaN,NaN,NaN


___
# Alright! Now let's just use our optimizer tools from above to generate some better lineups

In [22]:
out_players = list(master_projections.loc[(master_projections['Injury Indicator'] == 'IR') | (master_projections['Injury Indicator'] == 'O'), :]['Nickname'].values)
low_scorers = list(master_projections.loc[master_projections['FPPG'] < 0.80, :]['Nickname'].values)

all_removals = out_players + low_scorers
all_removals = list(set(all_removals))

In [23]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
# No stacking on this one

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.4):
    print(lineup, '\n')

 1. QB    Carson Wentz                  QB    PHI            3.078   7700.0$   
 2. RB    Saquon Barkley                RB    NYG            2.602   8800.0$   
 3. RB    David Johnson                 RB    ARI            2.532   7900.0$   
 4. WR    Tre'Quan Smith                WR    NO             1.429   4900.0$   
 5. WR    Michael Thomas                WR    NO             2.494   8800.0$   
 6. WR    John Brown                    WR    BAL            1.806   5400.0$   
 7. TE    Greg Olsen                    TE    CAR            2.008   6500.0$   
 8. D     Arizona Cardinals             D     ARI            4.125   4000.0$   
 9. FLEX  Alex Collins                  RB    BAL            1.817   6000.0$   

Fantasy Points 21.89
Salary 60000.0 

 1. QB    Carson Wentz                  QB    PHI            3.078   7700.0$   
 2. RB    Saquon Barkley                RB    NYG            2.602   8800.0$   
 3. RB    David Johnson                 RB    ARI            2.532   7900.0$   
 

 1. QB    Carson Wentz                  QB    PHI            3.078   7700.0$   
 2. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 3. RB    Tevin Coleman                 RB    ATL            2.007   6800.0$   
 4. WR    DeAndre Hopkins               WR    HOU            2.047   8600.0$   
 5. WR    Kenny Golladay                WR    DET            1.859   6400.0$   
 6. WR    Larry Fitzgerald              WR    ARI            1.678   5900.0$   
 7. TE    Evan Engram                   TE    NYG            1.681   5800.0$   
 8. D     Baltimore Ravens              D     BAL            3.119   4200.0$   
 9. FLEX  Alex Collins                  RB    BAL            1.817   6000.0$   

Fantasy Points 19.714
Salary 59800.0 

 1. QB    Drew Brees                    QB    NO             3.047   8500.0$   
 2. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 3. RB    Tevin Coleman                 RB    ATL            2.007   6800.0$   


In [24]:
exporter = CSVLineupExporter(optimizer.optimize(n=20))
exporter.export('submissions/Week-{}/fanduel-lineups-nostacks.csv'.format(current_week))

In [25]:
optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'WR'])

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.4):
    print(lineup, '\n')

 1. QB    Matthew Stafford              QB    DET            2.551   6800.0$   
 2. RB    Saquon Barkley                RB    NYG            2.602   8800.0$   
 3. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 4. WR    Kenny Golladay                WR    DET            1.859   6400.0$   
 5. WR    Devin Funchess                WR    CAR            1.649   5700.0$   
 6. WR    John Brown                    WR    BAL            1.806   5400.0$   
 7. TE    Greg Olsen                    TE    CAR            2.008   6500.0$   
 8. D     Arizona Cardinals             D     ARI            4.125   4000.0$   
 9. FLEX  David Johnson                 RB    ARI            2.532   7900.0$   

Fantasy Points 21.561
Salary 59900.0 

 1. QB    Eli Manning                   QB    NYG            2.724   6700.0$   
 2. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 3. RB    David Johnson                 RB    ARI            2.532   7900.0$   


 1. QB    Cam Newton                    QB    CAR            2.959   8600.0$   
 2. RB    Saquon Barkley                RB    NYG            2.602   8800.0$   
 3. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 4. WR    Kenny Golladay                WR    DET            1.859   6400.0$   
 5. WR    Larry Fitzgerald              WR    ARI            1.678   5900.0$   
 6. WR    Devin Funchess                WR    CAR            1.649   5700.0$   
 7. TE    Evan Engram                   TE    NYG            1.681   5800.0$   
 8. D     Baltimore Ravens              D     BAL            3.119   4200.0$   
 9. FLEX  Kerryon Johnson               RB    DET            1.831   6200.0$   

Fantasy Points 19.807
Salary 60000.0 

 1. QB    Cam Newton                    QB    CAR            2.959   8600.0$   
 2. RB    Saquon Barkley                RB    NYG            2.602   8800.0$   
 3. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   


In [26]:
exporter = CSVLineupExporter(optimizer.optimize(n=20))
exporter.export('submissions/Week-{}/fanduel-lineups-qbwr.csv'.format(current_week))

In [27]:
# Doing the second half with a RB + D stack

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['RB', 'D'])

    
# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
optimizer.set_min_salary_cap(58000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.4):
    print(lineup, '\n')

 1. QB    Carson Wentz                  QB    PHI            3.078   7700.0$   
 2. RB    Saquon Barkley                RB    NYG            2.602   8800.0$   
 3. RB    David Johnson                 RB    ARI            2.532   7900.0$   
 4. WR    Tre'Quan Smith                WR    NO             1.429   4900.0$   
 5. WR    Michael Thomas                WR    NO             2.494   8800.0$   
 6. WR    John Brown                    WR    BAL            1.806   5400.0$   
 7. TE    Greg Olsen                    TE    CAR            2.008   6500.0$   
 8. D     Arizona Cardinals             D     ARI            4.125   4000.0$   
 9. FLEX  Alex Collins                  RB    BAL            1.817   6000.0$   

Fantasy Points 21.89
Salary 60000.0 

 1. QB    Carson Wentz                  QB    PHI            3.078   7700.0$   
 2. RB    Saquon Barkley                RB    NYG            2.602   8800.0$   
 3. RB    David Johnson                 RB    ARI            2.532   7900.0$   
 

 1. QB    Drew Brees                    QB    NO             3.047   8500.0$   
 2. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 3. RB    Tevin Coleman                 RB    ATL            2.007   6800.0$   
 4. WR    Keenan Allen                  WR    LAC            1.901   7600.0$   
 5. WR    Kenny Golladay                WR    DET            1.859   6400.0$   
 6. WR    Corey Davis                   WR    TEN            1.694   6200.0$   
 7. TE    Evan Engram                   TE    NYG            1.681   5800.0$   
 8. D     Baltimore Ravens              D     BAL            3.119   4200.0$   
 9. FLEX  Alex Collins                  RB    BAL            1.817   6000.0$   

Fantasy Points 19.554
Salary 59900.0 

 1. QB    Drew Brees                    QB    NO             3.047   8500.0$   
 2. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 3. RB    Tevin Coleman                 RB    ATL            2.007   6800.0$   


In [28]:
exporter = CSVLineupExporter(optimizer.optimize(n=20))
exporter.export('submissions/Week-{}/fanduel-lineups-rbdst.csv'.format(current_week))

In [29]:
# QB + TE stack

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)

optimizer.load_players_from_csv('projections/Week-{}/master_projections.csv'.format(current_week))

# Removing certain players
for each_player in all_removals:
    optimizer.remove_player(each_player)

# Setting positional constraints
optimizer.set_players_with_same_position({'TE': 0})
optimizer.set_positions_for_same_team(['QB', 'TE'])

# Making more randomness
optimizer.set_deviation(0.06, 0.24)

# Setting a minimum salary cap
# optimizer.set_min_salary_cap(57000)

# Generating 20 lineups
for lineup in optimizer.optimize(n=20, max_exposure=.4):
    print(lineup, '\n')

 1. QB    Eli Manning                   QB    NYG            2.724   6700.0$   
 2. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 3. RB    David Johnson                 RB    ARI            2.532   7900.0$   
 4. WR    Michael Thomas                WR    NO             2.494   8800.0$   
 5. WR    Kenny Golladay                WR    DET            1.859   6400.0$   
 6. WR    John Brown                    WR    BAL            1.806   5400.0$   
 7. TE    Greg Olsen                    TE    CAR            2.008   6500.0$   
 8. D     Arizona Cardinals             D     ARI            4.125   4000.0$   
 9. FLEX  Evan Engram                   TE    NYG            1.681   5800.0$   

Fantasy Points 21.657
Salary 59900.0 

 1. QB    Eli Manning                   QB    NYG            2.724   6700.0$   
 2. RB    Saquon Barkley                RB    NYG            2.602   8800.0$   
 3. RB    David Johnson                 RB    ARI            2.532   7900.0$   


 1. QB    Cam Newton                    QB    CAR            2.959   8600.0$   
 2. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 3. RB    Tevin Coleman                 RB    ATL            2.007   6800.0$   
 4. WR    Keenan Allen                  WR    LAC            1.901   7600.0$   
 5. WR    Larry Fitzgerald              WR    ARI            1.678   5900.0$   
 6. WR    Devin Funchess                WR    CAR            1.649   5700.0$   
 7. TE    Greg Olsen                    TE    CAR            2.008   6500.0$   
 8. D     Baltimore Ravens              D     BAL            3.119   4200.0$   
 9. FLEX  Kerryon Johnson               RB    DET            1.831   6200.0$   

Fantasy Points 19.581
Salary 59900.0 

 1. QB    Cam Newton                    QB    CAR            2.959   8600.0$   
 2. RB    Ezekiel Elliott               RB    DAL            2.429   8400.0$   
 3. RB    Tevin Coleman                 RB    ATL            2.007   6800.0$   


In [30]:
exporter = CSVLineupExporter(optimizer.optimize(n=20))
exporter.export('submissions/Week-{}-qbte/fanduel-lineups-qbte.csv'.format(current_week))

FileNotFoundError: [Errno 2] No such file or directory: 'submissions/Week-11-qbte/fanduel-lineups-qbte.csv'